In [19]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import json
from datetime import *

StatementMeta(, e601d345-d732-4d1c-ba56-c5cfce136587, 21, Finished, Available, Finished)

In [20]:
#Leitura dos arquivos Parquet na pasta Novos
df = spark.read.parquet("Files/Cotacoes/Novos/*.parquet")

df.createOrReplaceTempView("df")

df = spark.sql("""
    SELECT
        cotacaoCompra AS Cotacao,
        CAST(dataHoraCotacao AS DATE) AS Data,
        moeda AS Moeda
    FROM
        df
    ORDER BY Data ASC
""").drop_duplicates(["Moeda", "Data"])
display(df)


StatementMeta(, e601d345-d732-4d1c-ba56-c5cfce136587, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e4b9afcf-01ad-4d64-8eb7-e40378778de3)

In [21]:
%%sql
-- Cria uma tabela no lake se não existir

CREATE TABLE IF NOT EXISTS Lakehouse_Silver.cotacoes(
    Cotacao DOUBLE,
    Data DATE,
    Moeda STRING
)

USING DELTA
PARTITIONED BY (Moeda)


StatementMeta(, e601d345-d732-4d1c-ba56-c5cfce136587, 23, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [22]:
# Executa o Merge na tabela cotacoes

df.createOrReplaceTempView("df_novos")

spark.sql("""
    MERGE INTO lakehouse_Silver.cotacoes AS e
    USING (
        SELECT
        Cotacao,
        Data,
        Moeda
    FROM
        df_novos
    ) as n

ON e.Moeda = n.Moeda
WHEN NOT MATCHED THEN
    INSERT (Cotacao, Data, Moeda)
    VALUES(n.Cotacao, n.Data, n.Moeda)
""")

StatementMeta(, e601d345-d732-4d1c-ba56-c5cfce136587, 24, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [23]:
%%sql
-- Contagem de linhas
SELECT COUNT(*) FROM Lakehouse_Silver.cotacoes

StatementMeta(, e601d345-d732-4d1c-ba56-c5cfce136587, 25, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [24]:

# Movimentação dos arquivos parquet da pasta Novos para Carregados

from notebookutils import mssparkutils

origem = f"abfss://e92bd821-b7e9-422c-b95e-71cd284d23fc@onelake.dfs.fabric.microsoft.com/03da6d4e-72c3-4cba-afc7-7914ef9d7913/Files/Cotacoes/Novos/"  
destino = f"abfss://e92bd821-b7e9-422c-b95e-71cd284d23fc@onelake.dfs.fabric.microsoft.com/03da6d4e-72c3-4cba-afc7-7914ef9d7913/Files/Cotacoes/Carregados/" 

if not mssparkutils.fs.exists(destino):
    mssparkutils.fs.mkdirs(destino)

arquivos = mssparkutils.fs.ls(origem)

for arquivo in arquivos:
    caminho_origem = arquivo.path
    nome_arquivo = arquivo.name
    caminho_destino = f"{destino}{nome_arquivo}"
    
    mssparkutils.fs.mv(caminho_origem, caminho_destino)

StatementMeta(, e601d345-d732-4d1c-ba56-c5cfce136587, 26, Finished, Available, Finished)